<!-- 99.4% validation accuracy
Less than 20k Parameters
You can use anything from above you want. 
Less than 20 Epochs
Have used BN, Dropout, a Fully connected layer, have used GAP. 
To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC. -->

# Step 1 
1. 99.4% validation accuracy
2. Less than 20k Parameters
3. You can use anything from above you want.
4. Less than 20 Epochs
5. Have used BN, Dropout, a Fully connected layer, have used GAP.
6. To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC.

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

/home/shivam13juna/Documents/virtual_envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CONVOLUTION BLOCK #1
        self.convblock1 = nn.Sequential( # 28x28 > 28x28 | RF 3 | jout=1
          nn.Conv2d(1, 10, 3, padding=1), 
          nn.ReLU(),
          nn.BatchNorm2d(10),
          nn.Dropout2d(0.05)
        )   
        self.convblock2 = nn.Sequential( # 28x28 > 28x28 | RF 5 | jout=1
          nn.Conv2d(10, 16, 3, padding=1), 
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.05)
        )
        # TRANSITIONAL BLOCK #1        
        self.pool1 = nn.MaxPool2d(2, 2) # 28x28 > 14x14 | RF 6 | jout=1
        self.convblock3 = nn.Sequential( # 14x14 > 14x14 | RF 6 | jout=2
          nn.Conv2d(16, 8, 1),
          nn.ReLU(),
          nn.BatchNorm2d(8),
          nn.Dropout2d(0.05)
        )

        # CONVOLUTION BLOCK #2
        self.convblock4 = nn.Sequential( # 14x14 > 14x14 | RF 10 | jout=2
          nn.Conv2d(8, 16, 3, padding=1), 
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.05)
        )

        self.convblock5 = nn.Sequential( # 14x14 > 14x14 | RF 14 | jout=2
          nn.Conv2d(16, 16, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.05)
        )
        self.convblock6 = nn.Sequential( # 14x14 > 14x14 | RF 18 | jout=2
          nn.Conv2d(16, 16, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout2d(0.05)
        )
        # TRANSITIONAL BLOCK #2    
        self.pool2 = nn.MaxPool2d(2, 2) # 14x14 > 7x7 | RF 20 | jout=2
        self.convblock7 = nn.Sequential( # 7x7 > 7x7 | RF 20 | jout=3
          nn.Conv2d(16, 8, 1),
          nn.ReLU(),
          nn.BatchNorm2d(8),
          nn.Dropout2d(0.05)
        )
        #CONVOLUTIONAL BLOCK #3
        self.convblock8 = nn.Sequential( #7x7 > 5x5 | RF 26 | jout=3
            nn.Conv2d(8, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.05)
        )
        self.convblock9 = nn.Sequential( #5x5 > 3x3 | RF 32 | jout=3
            nn.Conv2d(10, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05)
        )
        self.convblock10 = nn.Sequential( #3x3 > 3x3 | RF 38 | jout=3
            nn.Conv2d(16, 10, 1)
            #nn.ReLU(),
            #nn.BatchNorm2d(10),
            #nn.Dropout2d(0.1)
        )
        self.gap = nn.Sequential( #3x3 > 1x1 | RF 42 | jout=3
            nn.AvgPool2d(kernel_size=3)
        )



    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.pool1(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = self.convblock9(x)
        x = self.convblock10(x)
        x = self.gap(x)
      
        x = x.view(-1, 10)
        return F.log_softmax(x)



from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,456
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
         Dropout2d-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]             136
             ReLU-11            [-1, 8, 14, 14]               0
      BatchNorm2d-12            [-1, 8, 14, 14]              16
        Dropout2d-13            [-1, 8, 14, 14]               0
           Conv2d-14           [-1, 16,

/tmp/ipykernel_9918/3625294300.py:95: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
from torch.optim.lr_scheduler import StepLR

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



EPOCH: 0


Loss=0.3524477183818817 Batch_id=3749 Accuracy=96.49: 100%|██████████| 3750/3750 [00:24<00:00, 150.58it/s]    



Test set: Average loss: 0.0340, Accuracy: 9869/10000 (98.69%)

EPOCH: 1


Loss=0.07934898138046265 Batch_id=3749 Accuracy=98.21: 100%|██████████| 3750/3750 [00:28<00:00, 132.20it/s]   



Test set: Average loss: 0.0254, Accuracy: 9916/10000 (99.16%)

EPOCH: 2


...


Test set: Average loss: 0.0207, Accuracy: 9932/10000 (99.32%)

EPOCH: 18


Loss=0.04461562633514404 Batch_id=3749 Accuracy=99.41: 100%|██████████| 3750/3750 [00:25<00:00, 146.66it/s]   



Test set: Average loss: 0.0186, Accuracy: 9941/10000 (99.41%)

EPOCH: 19


Loss=0.01086433045566082 Batch_id=3749 Accuracy=99.44: 100%|██████████| 3750/3750 [00:25<00:00, 144.66it/s]   



Test set: Average loss: 0.0174, Accuracy: 9938/10000 (99.38%)

